In [315]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import run_weighting_iteration, run_weighting_scheme, normalize_weights


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(dem_vote, gop_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 3
    N_SIMS = 100000
    
    dem_moe = margin_of_error(n=n, p=dem_vote/100, interval_size=interval)
    gop_moe = margin_of_error(n=n, p=gop_vote/100, interval_size=interval)
    undecided = (100 - dem_vote - gop_vote) / 2

    dem_mean = dem_vote + undecided * 0.25
    dem_raw_moe = dem_moe * 100
    dem_allocate_undecided = undecided * 0.4
    dem_margin = dem_raw_moe + dem_allocate_undecided + GENERAL_POLLING_ERROR
    
    gop_mean = gop_vote + undecided * 0.25
    gop_raw_moe = gop_moe * 100
    gop_allocate_undecided = undecided * 0.4
    gop_margin = gop_raw_moe + gop_allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    dem_sigma = dem_margin / 100 / normed_sigma
    dem_sims = np.random.normal(dem_mean / 100, dem_sigma, N_SIMS)
    
    gop_sigma = gop_margin / 100 / normed_sigma
    gop_sims = np.random.normal(gop_mean / 100, gop_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] / 100 > sim[1] / 100 for sim in zip(dem_sims, gop_sims)]) / N_SIMS
    
    low, high = np.percentile(dem_sims - gop_sims, [20, 80]) * 100
    
    return {'mean': dem_mean - gop_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': dem_raw_moe + gop_raw_moe,
            'margin': (dem_margin + gop_margin) / 2,
            'sigma': (dem_sigma + gop_sigma) / 2,
            'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass, dem_name, gop_name=None):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result {} {}{} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) ({} {} likely to win)').format(dem_name,
                                                        '+' if mean > 0 else '',
                                                        mean,
                                                        first,
                                                        second,
                                                        n,
                                                        raw_moe,
                                                        margin,
                                                        sigma,
                                                        dem_name,
                                                        chance_pass))
    print('-')

In [316]:
survey = pd.read_csv('responses_processed.csv')

In [317]:
survey.shape

(1405, 34)

In [318]:
GA_CENSUS = {'survey_method': {'Online': 0.288, 'IVR': 0.712},
             'gender': {'Female': 0.511,
                        'Male': 0.483,
                        'Other': 0.006}, # Male-Female from 2010 US Census https://www.census.gov/prod/cen2010/briefs/c2010br-03.pdf, other from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5227946/
             'race': {'White, not Hispanic': 0.520,
                      'Black, non-Hispanic': 0.326,
                      'Hispanic': 0.099,
                      'Other': 0.055}, # https://www.census.gov/quickfacts/GA
             'education': {'Graduated from college': 0.183,
                           'Some college, no degree': 0.284,
                           'Completed graduate school': 0.091,
                           'Graduated from high school': 0.301,
                           'Less than high school': 0.141}, # https://statisticalatlas.com/state/Georgia/Educational-Attainment
             'income': {'Under $15,000': 0.1376,
                        'Between $15,000 and $49,999': 0.3524,
                        'Between $50,000 and $74,999': 0.1801,
                        'Between $75,000 and $99,999': 0.116,
                        'Between $100,000 and $150,000': 0.1207,
                        'Over $150,000': 0.0932}, # https://statisticalatlas.com/state/Georgia/Household-Income
             'age': {'18-34': 0.278,
                     '35-54': 0.359,
                     '55-64': 0.167,
                     '65 or older': 0.196}, # https://www.statista.com/statistics/910774/georgia-population-share-age-group/
             'vote2016': {'Donald Trump': 0.504,
                          'Hillary Clinton': 0.453,
                          'Other': 0.043},
             'vote2020': {'Joe Biden': 0.495,
                          'Donald Trump': 0.493,
                          'Other': 0.012},
             'loc_county': {'Fulton County, GA': 0.1,
                            'Cobb County, GA': 0.072,
                            'Gwinnett County, GA': 0.088,
                            'DeKalb County, GA': 0.0715,
                            'Another county in Georgia': 0.6685},
             'gss_trust': {'Can trust': 0.331,
                           'Can\'t be too careful': 0.669 }, # From GSS 2018 https://gssdataexplorer.norc.org/variables/441/vshow
             'gss_bible': {'Word of God': 0.41,
                           'Inspired word': 0.46,
                           'Book of fables': 0.13}, # From GSS 2018 https://gssdataexplorer.norc.org/variables/364/vshow (Region=South)
             'gss_spanking': {'Agree': 0.677, 'Disagree': 0.323},  # From GSS 2018 https://gssdataexplorer.norc.org/trends/Gender%20&%20Marriage?measure=spanking
             'social_fb': {'No': 0.31, 'Yes': 0.69}} # https://www.pewresearch.org/internet/fact-sheet/social-media/

weigh_on = ['loc_county', 'gender', 'race', 'education', 'income', 'age', 'vote2016', 'vote2020', 'survey_method']
            # experimental weights: 'gss_trust', 'gss_bible', 'gss_spanking', 'social_fb'
run_weighting_iteration(survey, weigh_on=weigh_on, census=GA_CENSUS)

## loc_county ##
Another county in Georgia    0.645715
Cobb County, GA              0.605109
DeKalb County, GA            2.130488
Fulton County, GA            0.678139
Gwinnett County, GA          0.940550
dtype: float64
ERROR: 0.35324570374958486
-
-
## gender ##
Female               1.075522
Male                 1.225079
Other                0.563080
Prefer not to say    1.136319
dtype: float64
ERROR: 0.1468926532157499
-
-
## race ##
Black, non-Hispanic    0.710423
Hispanic               2.623421
Other                  0.306188
White, not Hispanic    0.359968
dtype: float64
ERROR: 0.5862569586062975
-
-
## education ##
Completed graduate school     0.341333
Graduated from college        0.392915
Graduated from high school    1.156902
Less than high school         2.484733
Some college, no degree       0.624117
dtype: float64
ERROR: 0.5015363665753937
-
-
## income ##
Between $100,000 and $150,000    1.275756
Between $15,000 and $49,999      0.880656
Between $50,000 and $74,999     

{'errors': [0.35324570374958486,
  0.1468926532157499,
  0.5862569586062975,
  0.5015363665753937,
  0.1744838528302456,
  0.4720599263156817,
  0.06549888918025813,
  0.159555328306894,
  0.03458477143058395],
 'error_table': {'race': 1.6234208649704267,
  'education': 1.484732937472617,
  'loc_county': 1.1304875964632113,
  'age': 0.6536341994498072,
  'gender': 0.4369202697281025,
  'vote2020': 0.40707647991582774,
  'income': 0.35034967358962765,
  'vote2016': 0.1536527568970506,
  'survey_method': 0.03458477143058403},
 'weights': {'loc_county': Another county in Georgia    0.645715
  Cobb County, GA              0.605109
  DeKalb County, GA            2.130488
  Fulton County, GA            0.678139
  Gwinnett County, GA          0.940550
  dtype: float64,
  'gender': Female               1.075522
  Male                 1.225079
  Other                0.563080
  Prefer not to say    1.136319
  dtype: float64,
  'race': Black, non-Hispanic    0.710423
  Hispanic               2.62

In [319]:
 output = run_weighting_scheme(survey, iters=80, weigh_on=weigh_on, census=GA_CENSUS, verbose=1, early_terminate=False)

ITER 1/1 - initialization - ERROR 80
ITER 2/80 - weight loc_county - ERROR 2.4941144502106893
ITER 3/80 - weight gender - ERROR 2.1609862475221417
ITER 4/80 - weight race - ERROR 2.0424029568442914
ITER 5/80 - weight education - ERROR 1.9381156023622
ITER 6/80 - weight income - ERROR 1.8336572930900885
ITER 7/80 - weight age - ERROR 1.5107208813819102
ITER 8/80 - weight vote2016 - ERROR 1.5535454713731942
ITER 9/80 - weight vote2020 - ERROR 1.1911688923381336
ITER 10/80 - weight survey_method - ERROR 1.1042229806181492
ITER 11/80 - weight education - ERROR 1.2151874751430607
ITER 12/80 - weight gender - ERROR 0.8631898486045874
ITER 13/80 - weight race - ERROR 0.5204749845040186
ITER 14/80 - weight age - ERROR 0.4295020884410637
ITER 15/80 - weight income - ERROR 0.3748614428556033
ITER 16/80 - weight vote2020 - ERROR 0.3360774559707147
ITER 17/80 - weight vote2016 - ERROR 0.3995384647954907
ITER 18/80 - weight education - ERROR 0.4034842654590932
ITER 19/80 - weight gender - ERROR 0.2

In [320]:
survey = output['final_df']
_ = run_weighting_iteration(survey, weigh_on=weigh_on, census=GA_CENSUS)

## loc_county ##
Another county in Georgia    1.000461
Cobb County, GA              0.998992
DeKalb County, GA            1.001913
Fulton County, GA            1.000459
Gwinnett County, GA          0.998175
dtype: float64
ERROR: 0.000629643447751385
-
-
## gender ##
Female               0.999546
Male                 0.999221
Other                1.000960
Prefer not to say    1.000273
dtype: float64
ERROR: 0.0006054306278032818
-
-
## race ##
Black, non-Hispanic    1.000141
Hispanic               0.999636
Other                  1.001605
White, not Hispanic    0.998617
dtype: float64
ERROR: 0.0011147798264529805
-
-
## education ##
Completed graduate school     0.998090
Graduated from college        0.998390
Graduated from high school    1.001021
Less than high school         1.002489
Some college, no degree       1.000010
dtype: float64
ERROR: 0.001110741491618752
-
-
## income ##
Between $100,000 and $150,000    0.999612
Between $15,000 and $49,999      1.000261
Between $50,000 and $74

In [321]:
survey['lv_index'] = 0
survey.loc[(survey['lv_likely'] == 'Very likely'), 'lv_index'] += 1
survey.loc[(survey['lv_likely'] == 'Already voted'), 'lv_index'] += 1
survey.loc[(survey['lv_likely'] == 'Likely'), 'lv_index'] += 0.7
survey.loc[(survey['lv_likely'] == 'Somewhat likely'), 'lv_index'] += 0.4
survey.loc[(survey['lv_likely'] == 'Neither likely nor unlikely'), 'lv_index'] += 0.2
survey.loc[(survey['lv_likely'] == 'Somewhat unlikely'), 'lv_index'] += 0.1
survey.loc[(survey['lv_likely'] == 'Unlikely'), 'lv_index'] += 0.05

survey.loc[(survey['vote2020'] != 'Did not vote'), 'lv_index'] += 1

#survey.loc[(survey['lv_likely'] == 'Already voted'), 'lv_index'] += 0.2
#survey.loc[(survey['vote_method'] == 'I will vote on election day') & (survey['vote_method'] != 'Already voted'), 'lv_index'] += 0.2

max_pts = 2

survey['lv_index'].value_counts()



2.0    1189
1.0     164
1.7      25
1.4      19
0.0       5
0.4       3
Name: lv_index, dtype: int64

In [322]:
# https://www.pewresearch.org/methods/2016/01/07/measuring-the-likelihood-to-vote/
perry_gallup_loadings = {7: 0.83, 6: 0.63, 5: 0.59, 4: 0.4, 3: 0.34, 2: 0.23, 1: 0.13, 0: 0.11}
survey['lv_index'] = survey['lv_index'].apply(lambda l: perry_gallup_loadings[min(7, int(np.round(l * 7/max_pts)))])
survey['lv_weight'] = normalize_weights(survey['weight'] * survey['lv_index'])
survey['lv_index'].value_counts()

0.83    1189
0.40     164
0.63      25
0.59      19
0.11       5
0.13       3
Name: lv_index, dtype: int64

In [323]:
survey.to_csv('responses_processed_with_default_weights.csv', index=False)

In [324]:
print('## 2016 VOTE - DEMOGRAPHIC WEIGHTS ##')
options = ['Hillary Clinton', 'Donald Trump', 'Other', 'Did not vote']
survey_ = survey.loc[survey['vote2016'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote2016'].value_counts(normalize=True) * survey_.groupby('vote2016')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
data = calc_result(dem_vote=votes['Hillary Clinton'],
                   gop_vote=votes['Donald Trump'],
                   n=weighted_n)
data['dem_name'] = 'Clinton'
print_result(**data)

## 2016 VOTE - DEMOGRAPHIC WEIGHTS ##
Hillary Clinton    39.436882
Donald Trump       44.092402
Other               3.751829
Did not vote       12.718887
dtype: float64
Result Clinton -4.7 (80% CI: -12.7 to 3.3) (Weighted N=743) (raw_moe=4.6pts, margin=8.6pts, sigma=6.7pts) (Clinton 31.1% likely to win)
-


In [325]:
print('## 2016 VOTE - DEMOGRAPHIC WEIGHTS + 2020 RUNOFF LIKELY VOTERS ##')
votes = survey_['vote2016'].value_counts(normalize=True) * survey_.groupby('vote2016')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
data = calc_result(dem_vote=votes['Hillary Clinton'],
                   gop_vote=votes['Donald Trump'],
                   n=lv_weighted_n)
data['dem_name'] = 'Clinton'
print_result(**data)

## 2016 VOTE - DEMOGRAPHIC WEIGHTS + 2020 RUNOFF LIKELY VOTERS ##
Hillary Clinton    39.118504
Donald Trump       46.508904
Other               3.738293
Did not vote       10.634300
dtype: float64
Result Clinton -7.4 (80% CI: -15.0 to 0.2) (Weighted N=732) (raw_moe=4.7pts, margin=8.2pts, sigma=6.4pts) (Clinton 20.8% likely to win)
-


In [326]:
print('## 2020 PREZ VOTE - DEMOGRAPHIC WEIGHTS ##')
options = ['Joe Biden', 'Donald Trump', 'Other', 'Did not vote']
survey_ = survey.loc[survey['vote2020'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote2020'].value_counts(normalize=True) * survey_.groupby('vote2020')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
data = calc_result(dem_vote=votes['Joe Biden'],
                   gop_vote=votes['Donald Trump'],
                   n=weighted_n)
data['dem_name'] = 'Biden'
print_result(**data)

## 2020 PREZ VOTE - DEMOGRAPHIC WEIGHTS ##
Joe Biden       48.654448
Donald Trump    48.457865
Other            1.179502
Did not vote     1.708185
dtype: float64
Result Biden +0.2 (80% CI: -5.3 to 5.6) (Weighted N=743) (raw_moe=4.7pts, margin=5.9pts, sigma=4.6pts) (Biden 51.2% likely to win)
-


In [327]:
print('## 2020 PREZ VOTE - DEMOGRAPHIC WEIGHTS + 2020 RUNOFF LIKELY VOTERS ##')
votes = survey_['vote2020'].value_counts(normalize=True) * survey_.groupby('vote2020')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
data = calc_result(dem_vote=votes['Joe Biden'],
                   gop_vote=votes['Donald Trump'],
                   n=lv_weighted_n)
data['dem_name'] = 'Biden'
print_result(**data)

## 2020 PREZ VOTE - DEMOGRAPHIC WEIGHTS + 2020 RUNOFF LIKELY VOTERS ##
Joe Biden       47.045216
Donald Trump    51.195163
Other            1.136608
Did not vote     0.623013
dtype: float64
Result Biden -4.1 (80% CI: -9.5 to 1.2) (Weighted N=732) (raw_moe=4.7pts, margin=5.7pts, sigma=4.5pts) (Biden 25.6% likely to win)
-


In [328]:
print('## OSSOFF vs. PERDUE - DEMOGRAPHIC WEIGHTS ##')
options = ['Jon Ossoff', 'David Perdue', 'Undecided']
survey_ = survey.loc[survey['vote_ossoff_perdue'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_ossoff_perdue'].value_counts(normalize=True) * survey_.groupby('vote_ossoff_perdue')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
data = calc_result(dem_vote=votes['Jon Ossoff'],
                   gop_vote=votes['David Perdue'],
                   n=weighted_n)
data['dem_name'] = 'Ossoff'
print_result(**data)

## OSSOFF vs. PERDUE - DEMOGRAPHIC WEIGHTS ##
Jon Ossoff      48.627964
David Perdue    46.366253
Undecided        5.005784
dtype: float64
Result Ossoff +2.3 (80% CI: -3.7 to 8.1) (Weighted N=743) (raw_moe=4.7pts, margin=6.3pts, sigma=5.0pts) (Ossoff 62.5% likely to win)
-


In [329]:
print('## OSSOFF vs. PERDUE - DEMOGRAPHIC WEIGHTS + 2020 RUNOFF LIKELY VOTERS ##')
votes = survey_['vote_ossoff_perdue'].value_counts(normalize=True) * survey_.groupby('vote_ossoff_perdue')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
data = calc_result(dem_vote=votes['Jon Ossoff'],
                   gop_vote=votes['David Perdue'],
                   n=lv_weighted_n)
data['dem_name'] = 'Ossoff'
print_result(**data)

## OSSOFF vs. PERDUE - DEMOGRAPHIC WEIGHTS + 2020 RUNOFF LIKELY VOTERS ##
Jon Ossoff      46.597624
David Perdue    48.528411
Undecided        4.873965
dtype: float64
Result Ossoff -1.9 (80% CI: -7.8 to 4.0) (Weighted N=732) (raw_moe=4.7pts, margin=6.3pts, sigma=4.9pts) (Ossoff 39.3% likely to win)
-


In [330]:
print('## WARNOCK vs. LOEFFLER - DEMOGRAPHIC WEIGHTS ##')
options = ['Raphael Warnock', 'Kelly Loeffler', 'Undecided']
survey_ = survey.loc[survey['vote_warnock_loeffler'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_warnock_loeffler'].value_counts(normalize=True) * survey_.groupby('vote_warnock_loeffler')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
data = calc_result(dem_vote=votes['Raphael Warnock'],
                   gop_vote=votes['Kelly Loeffler'],
                   n=weighted_n)
data['dem_name'] = 'Warnock'
print_result(**data)

## WARNOCK vs. LOEFFLER - DEMOGRAPHIC WEIGHTS ##
Raphael Warnock    49.267266
Kelly Loeffler     47.117075
Undecided           3.615659
dtype: float64
Result Warnock +2.2 (80% CI: -3.5 to 7.7) (Weighted N=743) (raw_moe=4.7pts, margin=6.1pts, sigma=4.7pts) (Warnock 62.3% likely to win)
-


In [331]:
print('## WARNOCK vs. LOEFFLER - DEMOGRAPHIC WEIGHTS + 2020 LIKELY VOTER ##')
votes = survey_['vote_warnock_loeffler'].value_counts(normalize=True) * survey_.groupby('vote_warnock_loeffler')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
data = calc_result(dem_vote=votes['Raphael Warnock'],
                   gop_vote=votes['Kelly Loeffler'],
                   n=lv_weighted_n)
data['dem_name'] = 'Warnock'
print_result(**data)

## WARNOCK vs. LOEFFLER - DEMOGRAPHIC WEIGHTS + 2020 LIKELY VOTER ##
Raphael Warnock    47.290568
Kelly Loeffler     49.362623
Undecided           3.346809
dtype: float64
Result Warnock -2.1 (80% CI: -7.7 to 3.6) (Weighted N=732) (raw_moe=4.7pts, margin=6.0pts, sigma=4.7pts) (Warnock 38.0% likely to win)
-
